In [1]:
import numpy as np

In [2]:
def tt_to_tensor(factors):
    n = len(factors)
    s = ','.join([chr(ord('A') + i - 1) + chr(ord('a') + i) + chr(ord('A') + i) for i in range(1, n - 1)])
    s = 'aA,' + s + ',' + chr(ord('A') + n - 2) + chr(ord('a') + n - 1)
    T = np.einsum(s, *factors)
    return T

In [3]:
def truncated_svd(A, delta):
    u, s, v = np.linalg.svd(A, full_matrices=False)
    rank = len(s) - np.argmax(np.cumsum(s[:: -1]**2) > delta * s[0])
    return u[:, :rank], s[:rank], v[:rank, :], rank

In [4]:
def ttsvd(T, eps=1e-15):
    dims = list(T.shape)
    d = len(dims)
    S = [None for _ in range(d)]
    G = T.copy()
    rank = [1 for _ in range(d + 1)]

    for i in range(d - 1):
        G = np.reshape(G, (dims[i] * rank[i], -1))
        u, s, G, rank[i + 1] = truncated_svd(G, eps)
        if i == 0:
            S[i] = u
        else:
            S[i] = u.reshape(rank[i], dims[i], rank[i + 1])
        G = G * np.expand_dims(s, axis=1)

    S[-1] = G

    return S

In [5]:
def tt_sum(S1, S2):
    S3 = [None for _ in range(len(S1))]
    S3[0] = np.hstack((S1[0], S2[0]))

    for i in range(1, len(S1) - 1):
        x1, y1, z1 = S1[i].shape
        x2, y2, z2 = S2[i].shape

        S3[i] = np.zeros((x1 + x2, y1, z1 + z2))
        S3[i][: x1, : , : z1] = S1[i]
        S3[i][x1 :, : , z1 :] = S2[i]

    S3[-1] = np.vstack((S1[-1], S2[-1]))
    return S3

In [6]:
def squeeze(G, eps=1e-15):
    Q, R = np.linalg.qr(G[0], mode='reduced')
    G[0] = Q
    for i in range(1, len(G) - 1):
        G[i] = np.einsum('mk,krt->mrt', R, G[i])
        x, y, z = G[i].shape
        Q, R = np.linalg.qr(G[i].reshape(-1, z), mode='reduced')
        G[i] = Q.reshape(x, y, -1)

    u, s, G[-1], rank = truncated_svd(R @ G[-1], eps)
    u = u * np.expand_dims(s, axis=0)
    for i in range(len(G) - 2, 0, -1):
        G[i] = np.einsum('mrt,tk->mrk', G[i], u)
        x, n, r = G[i].shape
        G[i] = G[i].reshape(-1, n * rank)
        u, s, G[i], rank = truncated_svd(G[i], eps)
        G[i] = G[i].reshape(-1, n, r)
        u = u * np.expand_dims(s, axis=0)
    G[0] = G[0] @ u
    return G

# Tests

In [7]:
def generate_train(dims, ranks):
    train = [None for _ in range(len(ranks) + 1)]
    train[0] = np.random.normal(size=(dims[0], ranks[0]))
    for i in range(1, len(ranks)):
        train[i] = np.random.normal(size=(ranks[i - 1], dims[i], ranks[i]))
    train[-1] = np.random.normal(size=(ranks[-1], dims[-1]))
    return train

In [8]:
dims = [4, 8, 16, 32, 54]
ranks = [3, 3, 4, 2]
train = generate_train(dims, ranks)

In [9]:
T = tt_to_tensor(train)
train_new = ttsvd(T)
T_new = tt_to_tensor(train_new)
err = np.linalg.norm(T_new - T) / np.linalg.norm(T)

print(f'Train ranks: {[train_new[i].shape[-1] for i in range(len(train_new) - 1)]}')
print(f'Relative error: {err}')

Train ranks: [3, 3, 4, 2]
Relative error: 3.325554048700361e-15


In [19]:
def f(i, j, k, l, s):
    return 1 / (i + j + k + l + s + 1)

In [20]:
T = np.fromfunction(f, (16, 16, 16, 16, 16))

In [22]:
tt1 = ttsvd(T, eps=1e-4)
print(f'Train ranks, eps = {1e-4}: {[tt1[i].shape[-1] for i in range(len(tt1) - 1)]}')

tt2 = ttsvd(T, eps=1e-9)
print(f'Train ranks, eps = {1e-9}: {[tt2[i].shape[-1] for i in range(len(tt2) - 1)]}')

Train ranks, eps = 0.0001: [4, 4, 4, 4]
Train ranks, eps = 1e-09: [6, 7, 7, 6]


In [23]:
tt_of_sum = tt_sum(tt1, tt2)
print(f'Train ranks of sum, eps = {1e-5}: {[tt_of_sum[i].shape[-1] for i in range(len(tt_of_sum) - 1)]}')

tt_new = squeeze(tt_of_sum, eps = 1e-9)
print(f'Train ranks after compression: {[tt_new[i].shape[-1] for i in range(len(tt_new) - 1)]}')


Train ranks of sum, eps = 1e-05: [10, 11, 11, 10]
Train ranks after compression: [6, 7, 7, 6]
